<a href="https://colab.research.google.com/github/Avinash73-DA/opensky-weather-medallion-aws/blob/main/google_Colab/silver_push_Weather_flight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark findspark
import pandas as pd
import json
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("PySpark-in-Colab") \
    .getOrCreate()

print("SparkSession created Activated!")

SparkSession created Activated!


In [ ]:
df_plane = spark.read.format('json')\
          .option('multiline',True)\
          .load('/content/airplane_data_20250817_134507.json')

exploed_view = df_plane.select(explode('states').alias('state_data'))

columns = [
    'icao24', 'callsign', 'origin_country', 'time_position', 'last_contact',
    'longitude', 'latitude', 'baro_altitude', 'on_ground', 'velocity',
    'true_track', 'vertical_rate', 'sensors', 'geo_altitude', 'squawk',
    'spi', 'position_source'
]

df_plane = exploed_view\
.select(
    col('state_data').getItem(0).alias(columns[0]),
    col('state_data').getItem(1).alias(columns[1]),
    col('state_data').getItem(2).alias(columns[2]),
    col('state_data').getItem(3).cast('integer').alias(columns[3]),
    col('state_data').getItem(4).cast('integer').alias(columns[4]),
    col('state_data').getItem(5).cast('float').alias(columns[5]),
    col('state_data').getItem(6).cast('float').alias(columns[6]),
    col('state_data').getItem(7).cast('float').alias(columns[7]),
    col('state_data').getItem(8).cast('boolean').alias(columns[8]),
    col('state_data').getItem(9).cast('float').alias(columns[9]),
    col('state_data').getItem(10).cast('float').alias(columns[10]),
    col('state_data').getItem(11).cast('float').alias(columns[11]),
    col('state_data').getItem(12).alias(columns[12]),
    col('state_data').getItem(13).cast('float').alias(columns[13]),
    col('state_data').getItem(14).alias(columns[14]),
    col('state_data').getItem(15).cast('boolean').alias(columns[15]),
    col('state_data').getItem(16).cast('integer').alias(columns[16]),
    from_utc_timestamp(current_timestamp(), "Asia/Kolkata").alias("timestamp_sil")
)

In [ ]:
file_path = '/content/wether_data20250817_134528.json'

df_raw = spark.read.option("multiline", "true").json(file_path)

df_weather = df_raw.select(
    col("name").alias("city"),
    col("sys.country").alias("country"),
    col("coord.lat").alias("lat"),
    col("coord.lon").alias("lon"),
    col("weather")[0]["main"].alias("weather"),
    col("weather")[0]["description"].alias("description"),
    col("main.temp").alias("temp"),
    col("main.humidity").alias("humidity"),
    col("wind.speed").alias("wind_speed"),
    from_utc_timestamp(current_timestamp(), "Asia/Kolkata").alias("timestamp_sil")
)

df_weather.show(5)

+-----------+-------+-------+---------+-------+---------------+------+--------+----------+--------------------+
|       city|country|    lat|      lon|weather|    description|  temp|humidity|wind_speed|       timestamp_sil|
+-----------+-------+-------+---------+-------+---------------+------+--------+----------+--------------------+
|   New York|     US|40.7143|  -74.006| Clouds|overcast clouds|299.65|      76|      4.25|2025-08-21 21:59:...|
|Los Angeles|     US|34.0522|-118.2437|  Clear|      clear sky| 292.0|      82|      1.21|2025-08-21 21:59:...|
|    Chicago|     US|  41.85|   -87.65| Clouds|overcast clouds| 295.9|      94|      2.43|2025-08-21 21:59:...|
|    Houston|     US|29.7633| -95.3633|  Clear|      clear sky|301.46|      83|      0.57|2025-08-21 21:59:...|
|    Phoenix|     US|33.4484| -112.074|  Clear|      clear sky|300.35|      33|       1.6|2025-08-21 21:59:...|
+-----------+-------+-------+---------+-------+---------------+------+--------+----------+--------------

### Cross Join and Haversine Method

In [ ]:
df_Weather = df_weather.withColumnRenamed('lat','weather_lat')\
                      .withColumnRenamed('lon','weather_lon')

df_crossed = df_plane.crossJoin(broadcast(df_Weather)) \
    .filter(
        (col("weather_lat").between(col("latitude") - 2, col("latitude") + 2)) &
        (col("weather_lon").between(col("longitude") - 2, col("longitude") + 2))
    )

In [ ]:
print(df_crossed.count())

80498


In [ ]:
# Earth's radius in kilometers
R = lit(6371)

# Convert lat/lon to radians
lat1 = radians(col("latitude"))
lon1 = radians(col("longitude"))
lat2 = radians(col("weather_lat"))
lon2 = radians(col("weather_lon"))

dlon = lon2 - lon1
dlat = lat2 - lat1
a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * asin(sqrt(a))
haversine_expr = R * c

df_with_distance = df_crossed.withColumn("distance_km", haversine_expr)

window_Spec  = Window.partitionBy("icao24").orderBy("distance_km")
df_rank = df_with_distance.withColumn("rank", row_number().over(window_Spec))
df_nearest = df_rank.filter(col("rank") == 1)

df_nearest.show()

+------+--------+--------------+-------------+------------+---------+--------+-------------+---------+--------+----------+-------------+-------+------------+------+-----+---------------+--------------------+------------+-------+-----------+-----------+-------+-----------+------+--------+----------+--------------------+------------------+----+
|icao24|callsign|origin_country|time_position|last_contact|longitude|latitude|baro_altitude|on_ground|velocity|true_track|vertical_rate|sensors|geo_altitude|squawk|  spi|position_source|       timestamp_sil|        city|country|weather_lat|weather_lon|weather|description|  temp|humidity|wind_speed|       timestamp_sil|       distance_km|rank|
+------+--------+--------------+-------------+------------+---------+--------+-------------+---------+--------+----------+-------------+-------+------------+------+-----+---------------+--------------------+------------+-------+-----------+-----------+-------+-----------+------+--------+----------+-----------